In [1]:
import psycopg2
import pandas as pd
import requests

In [2]:
import base64
from pyDes import des, ECB, PAD_PKCS5
import hashlib

import time
import urllib.parse
from datetime import datetime, timedelta

import xml.etree.ElementTree as ET

In [3]:
def DESEnCode(mystr, deskey):
    mytext = mystr.encode('utf-8')
    key = deskey.encode('utf-8')
    iv = deskey.encode('utf-8')
    
    # DES Encryption
    k = des(key, ECB, IV=iv, pad=None, padmode=PAD_PKCS5)
    cryptoText = k.encrypt(mytext)
    
    # Encoding to base64
    encoded = base64.b64encode(cryptoText).decode('utf-8')
    return encoded

def BuildMD5(mystr):
    h = hashlib.md5()
    h.update(mystr.encode('utf-8'))
    data = h.hexdigest()
    return data

In [4]:
defaultSPSecretKey = "494DA5A7FC954AA4973456A08609C2F9"
defaultSPMd5key    = "GgaIMaiNNtg"
defaultSPEncrypKey = "g9G16nTs"

# Get the current date
today = datetime.now()

# Calculate the date one year ago from today
one_year_ago = today - timedelta(days=365)

# Format the dates as strings in the format "YYYYMMDDHHMMSS"
dateFrom = one_year_ago.strftime("%Y%m%d%H%M%S")
dateTo = today.strftime("%Y%m%d%H%M%S")

In [5]:
timeStr = time.strftime("%Y%m%d%H%M%S")
qs = "method=GetAllBetDetailsForTimeInterval"
qs = qs + "&Key=" + '494DA5A7FC954AA4973456A08609C2F9'
qs = qs + "&Time=" + timeStr
qs = qs + "&FromTime=" + dateFrom
qs = qs + "&ToTime=" + dateTo

In [6]:
q = DESEnCode(qs, defaultSPEncrypKey)
sig = BuildMD5(qs + defaultSPMd5key + timeStr + defaultSPSecretKey)

data = urllib.parse.urlencode({'q': q, 's': sig})

In [97]:
url = 'https://om6wq.mocklab.io/json'

res = requests.post(url, data=data)

In [100]:
# Parse the XML response
root = ET.fromstring(res.text)

In [102]:
# Access elements in the XML and extract data
error_msg_id = root.find('ErrorMsgId').text
error_msg = root.find('ErrorMsg').text

In [115]:
# Access nested elements
bet_details = []
for b in root.findall('.//BetDetail'):
    # Create a dictionary to store the data for each bet detail
    bet_data = {
        'BetTime': b.find('BetTime').text,
        'PayoutTime': b.find('PayoutTime').text,
        'BetTime': b.find('BetTime').text,
        'PayoutTime': b.find('PayoutTime').text,
        'Username': b.find('Username').text,
        'HostID': b.find('HostID').text,
        'Detail': b.find('Detail').text,
        'GameID': b.find('GameID').text,
        'Round': b.find('Round').text,
        'Set': b.find('Set').text,
        'BetID': b.find('BetID').text,
        'BetAmount': b.find('BetAmount').text,
        'Rolling': b.find('Rolling').text,
        'ResultAmount': b.find('ResultAmount').text,
        'Balance': b.find('Balance').text,
        'GameType': b.find('GameType').text,
        'BetType': b.find('BetType').text,
        'BetSource': b.find('BetSource').text,
        'TransactionID': b.find('TransactionID').text,
        'GameResult': b.find('GameResult').text,
        'State': b.find('State').text,
    }
    bet_details.append(bet_data)

# Create a dictionary to store the entire response
history = {
    'ErrorMsgID': error_msg_id,
    'ErrorMsg': error_msg,
    'BetDetailList': bet_details,
}

# You can then work with the data in the 'history_response' dictionary
print(history)

{'ErrorMsgID': '1001', 'ErrorMsg': 'Error message here', 'BetDetailList': [{'BetTime': '2023-07-31T10:15:30Z', 'PayoutTime': '2023-07-31T11:20:45Z', 'Username': 'user123', 'HostID': '1', 'Detail': 'Some betting details here', 'GameID': 'G12345', 'Round': '5', 'Set': '1', 'BetID': '987654321', 'BetAmount': '100.50', 'Rolling': '20.75', 'ResultAmount': '120.25', 'Balance': '2000.00', 'GameType': 'Slots', 'BetType': '1', 'BetSource': '2', 'TransactionID': '123456789', 'GameResult': 'Win', 'State': 'true'}, {'BetTime': '2023-07-31T10:15:30Z', 'PayoutTime': '2023-07-31T11:20:45Z', 'Username': 'user123', 'HostID': '1', 'Detail': 'Some betting details here', 'GameID': 'G12345', 'Round': '5', 'Set': '1', 'BetID': '987654513', 'BetAmount': '100.50', 'Rolling': '20.75', 'ResultAmount': '120.25', 'Balance': '2000.00', 'GameType': 'Fishing', 'BetType': '1', 'BetSource': '2', 'TransactionID': '123456789', 'GameResult': 'Win', 'State': 'true'}]}


# We don't save game result?

In [104]:
conn = psycopg2.connect(
    database="collectorDB",
    host='172.17.0.1',
    user='airflow',
    password='airflow',
    port='5432'
)

In [105]:
cursor = conn.cursor()

In [114]:
for b in history['BetDetailList']:
    print(b)


{'BetTime': '2023-07-31T10:15:30Z', 'PayoutTime': '2023-07-31T11:20:45Z'}
{'BetTime': '2023-07-31T10:15:30Z', 'PayoutTime': '2023-07-31T11:20:45Z'}


In [116]:
sql = """
    INSERT INTO simpleplay_wager(bet_id, bet_time, payout_time, username, host_id, detail, game_id,
        round, set, bet_amount, rolling, result_amount, balance, game_type,
        bet_type, bet_source, transaction_id, state)
    VALUES(%s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s);
"""

for b in history['BetDetailList']:
    cursor.execute(sql, (b['BetID'], b['BetTime'], b['PayoutTime'], b['Username'], b['HostID'], b['Detail'], b['GameID'], 
                         b['Round'], b['Set'], b['BetAmount'], b['Rolling'], b['ResultAmount'], b['Balance'], b['GameType'], 
                         b['BetType'], b['BetSource'], b['TransactionID'], b['State']))
 

In [117]:
conn.commit()

In [118]:
cursor.close()

# Fetching data

In [49]:
cursor.execute("SELECT * FROM simpleplay_wager")

In [50]:
records = cursor.fetchall()

In [60]:
cursor.description

(Column(name='id', type_code=20),
 Column(name='bet_id', type_code=20),
 Column(name='bet_time', type_code=1114),
 Column(name='payout_time', type_code=1114),
 Column(name='username', type_code=1043),
 Column(name='host_id', type_code=20),
 Column(name='detail', type_code=1043),
 Column(name='game_id', type_code=1043),
 Column(name='round', type_code=23),
 Column(name='set', type_code=23),
 Column(name='bet_amount', type_code=1700),
 Column(name='rolling', type_code=1700),
 Column(name='result_amount', type_code=1700),
 Column(name='balance', type_code=1700),
 Column(name='game_type', type_code=1043),
 Column(name='bet_type', type_code=23),
 Column(name='bet_source', type_code=23),
 Column(name='transaction_id', type_code=20),
 Column(name='state', type_code=16),
 Column(name='create_at', type_code=1114),
 Column(name='update_at', type_code=1114))

In [51]:
df = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])

In [52]:
df.describe()

,id,bet_id,bet_time,payout_time,host_id,round,set,bet_type,bet_source,transaction_id,create_at,update_at
count,1000.000000,998.000000,1000,1000,1000.000000,1000.00000,1000.000000,1000.00000,1000.000000,1000.000000,1000,1000
mean,500.500000,525.356713,1997-09-12 13:06:55.130000,1996-12-13 15:33:27.022000,522.565000,505.27100,525.490000,494.53100,511.931000,506.436000,1997-03-08 07:00:54.086000,1996-03-03 09:44:32.483999872
min,1.000000,1.000000,1970-02-18 01:50:17,1970-01-30 18:57:19,2.000000,1.00000,6.000000,2.00000,1.000000,4.000000,1970-01-19 03:46:20,1970-01-30 07:15:51
25%,250.750000,271.000000,1983-09-18 02:19:54.750000,1983-12-13 21:49:18,289.750000,242.75000,270.750000,234.75000,275.750000,263.500000,1983-08-02 02:10:39.250000,1982-03-24 12:58:20
50%,500.500000,538.000000,1998-08-04 19:39:31,1997-02-02 01:16:05,515.000000,486.50000,532.500000,485.50000,517.500000,482.500000,1997-10-28 10:49:56.500000,1995-04-18 23:45:19.500000
75%,750.250000,778.000000,2011-07-06 23:12:45.500000,2009-10-25 18:29:03.500000,762.250000,763.50000,780.250000,744.25000,757.000000,761.000000,2010-12-11 17:04:19,2010-04-08 07:24:46.500000
max,1000.000000,1024.000000,2023-07-16 08:55:19,2023-07-12 09:20:36,1023.000000,1024.00000,1023.000000,1022.00000,1024.000000,1023.000000,2023-07-26 12:51:55,2023-06-12 13:25:47
std,288.819436,298.342655,NaN,NaN,286.645592,298.13596,297.272536,295.38103,289.478573,292.230749,NaN,NaN


In [53]:
df.drop_duplicates(subset = 'bet_id', inplace=True)
df.dropna(subset = ['bet_id'], inplace=True)

In [54]:
df.describe()

,id,bet_id,bet_time,payout_time,host_id,round,set,bet_type,bet_source,transaction_id,create_at,update_at
count,610.000000,610.000000,610,610,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610,610
mean,420.395082,521.152459,1998-06-12 23:10:25.991803392,1997-01-17 17:07:47.957377024,521.968852,501.144262,517.037705,492.559016,505.444262,504.865574,1997-01-14 17:03:01.419672192,1995-07-19 16:54:34.522950784
min,1.000000,1.000000,1970-02-25 11:01:02,1970-01-30 18:57:19,2.000000,4.000000,9.000000,5.000000,1.000000,4.000000,1970-01-19 03:46:20,1970-02-08 11:01:51
25%,172.250000,262.250000,1983-12-27 23:07:35.500000,1984-03-20 21:12:14.750000,293.250000,240.750000,262.250000,222.250000,265.000000,253.000000,1983-05-08 15:54:04.500000,1982-04-11 11:25:16.750000
50%,368.500000,536.500000,1999-03-27 05:11:27.500000,1997-01-30 02:46:19.500000,512.000000,483.000000,521.000000,493.000000,505.500000,483.500000,1997-03-31 19:44:32.500000,1994-04-21 05:03:10.500000
75%,654.500000,775.750000,2012-05-31 22:49:27,2009-09-24 16:00:41.750000128,757.750000,755.750000,771.750000,735.500000,759.250000,760.750000,2011-02-10 21:07:10.500000,2008-09-08 11:25:05.500000
max,1000.000000,1024.000000,2023-06-30 23:03:19,2023-07-12 09:20:36,1023.000000,1023.000000,1023.000000,1019.000000,1019.000000,1023.000000,2023-07-26 12:51:55,2023-06-12 13:25:47
std,284.957318,299.048889,NaN,NaN,285.311452,293.765870,301.749339,296.003673,291.066812,294.458901,NaN,NaN


In [55]:
df.to_csv('simpleplay.csv')

In [58]:
rdf = pd.read_csv('simpleplay.csv')
print(rdf.head())

   Unnamed: 0  id  bet_id             bet_time          payout_time username  \
0           0   1   271.0  2008-11-14 18:05:56  1973-11-12 14:20:02    kwamg   
1           1   6   574.0  1990-09-20 17:46:16  2003-06-17 10:17:01    nkrwj   
2           2   7   830.0  1989-12-12 21:48:00  1974-09-28 08:27:34    vfxsp   
3           3  10   820.0  2003-12-07 09:06:47  1989-05-03 05:14:18    fawuh   
4           4  11  1021.0  2018-04-04 02:52:38  2020-08-10 00:04:19    zinwc   

   host_id detail game_id  round  ...  rolling  result_amount  balance  \
0      530  xqogh   idwjo    189  ...      218            915      865   
1       63  epfqg   txvcc    423  ...      714            815       72   
2      415  fqpgx   muirb   1007  ...      867            116      468   
3     1015  rgsmq   ldsdp    168  ...      416            365      139   
4      294  qhupq   wxtif    833  ...      503            282      569   

   game_type  bet_type bet_source  transaction_id  state            create